In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import json
import pandas as pd
import json
import yahoo_fin.stock_info as si
import random
import pyttsx3
import numpy as np

In [2]:
# open list of user agents
with open("../../../Data/user_agent_list.json", "r") as f:
    user_agent_list = json.load(f)

In [3]:
# tickers
with open("../relevant_trading_212_tickers.json", "r") as f:
    ticker_list = json.load(f)

In [14]:
ticker_df_list = []
n = 0
m = 0

for ticker in ticker_list[3000:]:
    try:
        

        financial_data = si.get_financials(ticker, yearly=False)
        ebit = financial_data["quarterly_income_statement"].loc["ebit"]
        total_assets = financial_data["quarterly_balance_sheet"].loc["totalAssets"]
        stockholder_equity = financial_data["quarterly_balance_sheet"].loc["totalStockholderEquity"]
        share_rep = financial_data["quarterly_cash_flow"].loc["repurchaseOfStock"]

        ticker_df = pd.DataFrame({
            "ticker" : ticker,
            "ebit_date" : ebit.index,
            "ebit" : ebit.values,
            "ass_tot_date" : total_assets.index,
            "ass_tot" : total_assets.values,
            "book_equ_date" : stockholder_equity.index,
            "book_equ" : stockholder_equity.values,
            "share_rep_date" : share_rep.index,
            "share_rep" : share_rep.values
        })

        ticker_df_list.append(ticker_df)
        n += 1
        print("n: " + str(n))
    except:
        m += 1
        print("m: " + str(m))
        pass
    
text = "Ich bin fertig du Sneck"

speaker = pyttsx3.init()
speaker.say(text)
speaker.runAndWait()



n: 1
m: 1
m: 2
n: 2
m: 3
m: 4
m: 5
n: 3
m: 6
m: 7
n: 4
n: 5
m: 8
n: 6
n: 7
n: 8
n: 9
m: 9
m: 10
m: 11
m: 12
n: 10
m: 13
m: 14
m: 15
n: 11
m: 16
n: 12
n: 13
n: 14
m: 17
n: 15
m: 18
m: 19
n: 16
n: 17
m: 20
m: 21
n: 18
n: 19
m: 22
m: 23
m: 24
m: 25
n: 20
n: 21
n: 22
m: 26
n: 23
m: 27
n: 24
n: 25
m: 28
m: 29
m: 30
n: 26
n: 27
n: 28
n: 29
m: 31
n: 30
n: 31
m: 32
n: 32
m: 33
n: 33
m: 34
m: 35
n: 34
m: 36
n: 35
m: 37
m: 38
n: 36
m: 39
n: 37
m: 40
n: 38
m: 41
n: 39
m: 42
n: 40
n: 41
n: 42
n: 43
m: 43
n: 44
n: 45
m: 44
n: 46
m: 45
m: 46
n: 47
n: 48
m: 47
m: 48
n: 49
m: 49
m: 50
n: 50
m: 51
m: 52
n: 51
n: 52
m: 53
n: 53
m: 54
m: 55
n: 54
m: 56
m: 57
n: 55
n: 56
m: 58
n: 57
n: 58
m: 59
n: 59
n: 60
n: 61
m: 60
n: 62
n: 63
m: 61
n: 64
n: 65
n: 66
n: 67
n: 68
n: 69
n: 70
m: 62
m: 63
n: 71
n: 72
n: 73
n: 74
n: 75
m: 64
n: 76
m: 65
n: 77
n: 78
n: 79
m: 66
n: 80
m: 67
m: 68
m: 69
n: 81
n: 82
n: 83
m: 70
m: 71
n: 84
n: 85
m: 72
m: 73
m: 74
n: 86
n: 87
m: 75
n: 88
m: 76
n: 89
m: 77
n: 90
n: 91
m: 78
m: 7

In [15]:
all_ticker_df = pd.concat(ticker_df_list)
all_ticker_df.to_csv("all_ticker_funds_from_3000.csv")

In [4]:
bis_1000 = pd.read_csv("all_ticker_funds_0_1000.csv")
tau_bis_3000 = pd.read_csv("all_ticker_funds_1000_3000.csv")
drei_tau_bis_ende = pd.read_csv("all_ticker_funds_from_3000.csv")

In [5]:
dfs = [bis_1000, tau_bis_3000, drei_tau_bis_ende]
all_funds_df = pd.concat(dfs)

In [6]:
len(all_funds_df.ticker.unique())

2836

In [7]:
for e, a, b, s in zip(all_funds_df.ebit_date, all_funds_df.ass_tot_date, all_funds_df.book_equ_date, all_funds_df.share_rep_date):
    if e == a == b == s:
        pass
    else:
        print("nope")
        
# date is equal

In [8]:
funds_data = all_funds_df[["ebit_date", "ticker", "ass_tot", "book_equ", "share_rep"]]

In [9]:
funds_data.columns = ["date", "ticker", "assets", "book_equity", "shares_rep"]

In [10]:
funds_tickers = funds_data.ticker.unique()
funds_dates = funds_data.date.unique()

## Add shares outstanding to the df

In [11]:
shares_out_df = pd.read_csv("shares_outstanding_df.csv", index_col=0)

In [12]:
shares_out_df.head()

,date,ticker,shares_outstanding
0,"Nov. 6, 2020",EMCF,2709000.0
1,"Sept. 30, 2020",EMCF,2709000.0
2,"Aug. 11, 2020",EMCF,2709000.0
3,"June 30, 2020",EMCF,2709000.0
4,"May 12, 2020",EMCF,2709000.0


In [13]:
# get the date in following format:  yyyy-mm-dd
dates = shares_out_df.date.tolist()

# create dict
months_dict = {
    "Nov." : "-11-",
    "Sept." : "-09-",
    "Aug." : "-08-",
    "June" : "-06-",
    "May" : "-05-",
    "March" : "-03-",
    "April" : "-04-",
    "Feb." : "-02-",
    "Oct." : "-10-",
    "July" : "-07-", 
    "Dec." : "-12-",
    "Jan." : "-01-"
}

days_dict = {
    "1" : "01",
    "2" : "02",
    "3" : "03",
    "4": "04",
    "5" : "05",
    "6" : "06",
    "7" : "07",
    "8" : "08",
    "9" : "09"
}

In [14]:
splitted_date = [date.split(",") for date in shares_out_df.date.tolist()]

In [15]:
shares_out_df["month_day"] = [i[0] for i in splitted_date]
shares_out_df["year"] = [i[1] for i in splitted_date]

In [16]:
splitted_month = [date.split() for date in shares_out_df.month_day.tolist()]

In [17]:
shares_out_df["month"] = [i[0] for i in splitted_month]
shares_out_df["day"] = [i[1] for i in splitted_month]

In [18]:
shares_out_df.reset_index(drop = True, inplace = True)

In [19]:
shares_out_df["year"] = [i.strip() for i in shares_out_df["year"].tolist()]

In [20]:
shares_out_df.head()

,date,ticker,shares_outstanding,month_day,year,month,day
0,"Nov. 6, 2020",EMCF,2709000.0,Nov. 6,2020,Nov.,6
1,"Sept. 30, 2020",EMCF,2709000.0,Sept. 30,2020,Sept.,30
2,"Aug. 11, 2020",EMCF,2709000.0,Aug. 11,2020,Aug.,11
3,"June 30, 2020",EMCF,2709000.0,June 30,2020,June,30
4,"May 12, 2020",EMCF,2709000.0,May 12,2020,May,12


In [21]:
dates = []

for day, month, year in zip(shares_out_df.day, shares_out_df.month, shares_out_df.year):
    m = months_dict[month]
    if day in days_dict.keys():
        d = days_dict[day]
    else:
        d = day
    
    date = str(year + m + d)
    
    dates.append(date)

In [22]:
# add the correct date_column

shares_out_df["rel_date"] = dates

In [23]:
rel_so_df = shares_out_df[["rel_date", "ticker", "shares_outstanding"]]

In [24]:
common_tickers = set.intersection(set(funds_data.ticker), set(rel_so_df.ticker))
common_dates = set.intersection(set(funds_data.date), set(rel_so_df.rel_date))

In [25]:
rel_so_df = rel_so_df[(rel_so_df.rel_date.isin(common_dates)) & (rel_so_df.ticker.isin(common_tickers))]

In [26]:
rel_so_df.reset_index(drop = True, inplace = True)
len(rel_so_df)

24862

In [27]:
funds_data.reset_index(drop = True, inplace = True)

In [28]:
# list of the shares outstanding at the relevant date

s_o_list = []

for d, t in zip(funds_data.date, funds_data.ticker):
    try:
        s_o = rel_so_df[(rel_so_df.ticker == t) & (rel_so_df.rel_date == d)]["shares_outstanding"].item()
        s_o_list.append(s_o)
       
    except:
        s_o_list.append(np.nan)
    

In [29]:
funds_data["shares_outstanding"] = s_o_list

C:\Users\Leonhard\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
funds_data

,date,ticker,assets,book_equity,shares_rep,shares_outstanding
0,2020-09-30,ARCC,1.495000e+10,6.963000e+09,NaN,422620000.0
1,2020-06-30,ARCC,1.451700e+10,6.691000e+09,NaN,423000000.0
2,2020-03-31,ARCC,1.580600e+10,6.583000e+09,-100000000.0,423000000.0
3,2019-12-31,ARCC,1.490500e+10,7.467000e+09,-100000000.0,431000000.0
4,2020-09-30,AEIS,1.642519e+09,7.681520e+08,-4331000.0,38270000.0
...,...,...,...,...,...,...
11339,2019-12-31,PRSP,6.189000e+09,2.206000e+09,-16000000.0,161530000.0
11340,2020-09-30,WTRG,1.339949e+10,4.635753e+09,-2000.0,245270000.0
11341,2020-06-30,WTRG,1.302841e+10,4.635534e+09,-4000.0,245150000.0
11342,2020-03-31,WTRG,1.292899e+10,4.613163e+09,-4339000.0,245040000.0


In [31]:
# drop data when shares outstanding is missing
rel_funds = funds_data.dropna(subset=["shares_outstanding"])

In [32]:
rel_funds.reset_index(drop = True, inplace = True)

In [33]:
# import the price data
price_data = pd.read_csv("../relevant_price_data_from_2016.csv", index_col=0)

In [34]:
price_data.head()

,date,ticker,adjclose
2016-01-04,2016-01-04,DDD,9.83
2016-01-05,2016-01-05,DDD,9.69
2016-01-06,2016-01-06,DDD,9.10
2016-01-07,2016-01-07,DDD,8.58
2016-01-08,2016-01-08,DDD,8.37


In [59]:
len(rel_funds)

10544

In [ ]:
prices = []

n = 0
m = 0

%timeit
for d, t in zip(rel_funds.date[1000:], rel_funds.ticker[1000:]):
    try:
        price = price_data[(price_data.ticker == t) & (price_data.date == d)]["adjclose"].item()
        prices.append(price)
        n += 1
        print("n: ", str(n))
    except:
        prices.append(np.nan)
        m += 1
        print("m: ", str(m))

n:  1
n:  2
n:  3
n:  4
n:  5
n:  6
n:  7
n:  8
n:  9
n:  10
n:  11
n:  12
n:  13
n:  14
n:  15
n:  16
n:  17
m:  1
n:  18
n:  19
n:  20
n:  21
n:  22
n:  23
n:  24
n:  25
n:  26
n:  27
n:  28
n:  29
n:  30
n:  31
n:  32
n:  33
n:  34
n:  35
n:  36
n:  37
n:  38
n:  39
n:  40
m:  2
m:  3
m:  4
m:  5
n:  41
n:  42
n:  43
n:  44
n:  45
n:  46
n:  47
n:  48
n:  49
n:  50
n:  51
n:  52
n:  53
n:  54
n:  55
n:  56
n:  57
n:  58
n:  59
n:  60
n:  61
m:  6
m:  7
n:  62
n:  63
n:  64
n:  65
n:  66
n:  67
n:  68
n:  69
n:  70
n:  71
n:  72
n:  73
n:  74
n:  75
n:  76
n:  77
n:  78
n:  79
n:  80
n:  81
n:  82
n:  83
n:  84
n:  85
n:  86
n:  87
n:  88
n:  89
n:  90
n:  91
n:  92
n:  93
n:  94
n:  95
n:  96
n:  97
n:  98
n:  99
n:  100
n:  101
n:  102
n:  103
n:  104
n:  105
n:  106
n:  107
n:  108
n:  109
n:  110
n:  111
n:  112
n:  113
n:  114
n:  115
n:  116
n:  117
n:  118
n:  119
n:  120
n:  121
n:  122
n:  123
n:  124
m:  8
m:  9
m:  10
n:  125
n:  126
n:  127
n:  128
n:  129
n:  130
n:  131

In [63]:
prices_to_1000 = prices

In [58]:
rel_funds

,date,ticker,assets,book_equity,shares_rep,shares_outstanding
0,2020-09-30,ARCC,1.495000e+10,6.963000e+09,NaN,422620000.0
1,2020-06-30,ARCC,1.451700e+10,6.691000e+09,NaN,423000000.0
2,2020-03-31,ARCC,1.580600e+10,6.583000e+09,-100000000.0,423000000.0
3,2019-12-31,ARCC,1.490500e+10,7.467000e+09,-100000000.0,431000000.0
4,2020-09-30,AEIS,1.642519e+09,7.681520e+08,-4331000.0,38270000.0
...,...,...,...,...,...,...
10539,2019-12-31,PRSP,6.189000e+09,2.206000e+09,-16000000.0,161530000.0
10540,2020-09-30,WTRG,1.339949e+10,4.635753e+09,-2000.0,245270000.0
10541,2020-06-30,WTRG,1.302841e+10,4.635534e+09,-4000.0,245150000.0
10542,2020-03-31,WTRG,1.292899e+10,4.613163e+09,-4339000.0,245040000.0
